In [ ]:
import os
import duckdb
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# this works with MIMIC-III demo 
# https://physionet.org/content/mimiciii-demo/1.4/

# If you want to persist the database, uncomment the following line and set the path instead of in memory
# you may need to do this if you are working with large data or don't have enough RAM
# db_path = './data/db/mimic.db'
db_path = ':memory:'
extract_dir = './data/mimic-demo'
con = duckdb.connect(db_path)

def detect_date_columns(sample_df):
    date_cols = []
    for col in sample_df.columns:
        try:
            parsed = pd.to_datetime(sample_df[col], errors='coerce')
            non_null = parsed.notnull().sum()
            if non_null / len(parsed) > 0.8:  # at least 80% parsable
                date_cols.append(col)
        except Exception:
            continue
    return date_cols

for file_name in os.listdir(extract_dir):
    if file_name.endswith('.csv'):
        file_path = os.path.join(extract_dir, file_name)
        table_name = os.path.splitext(file_name)[0]

        try:
            sample = pd.read_csv(file_path, nrows=100)
            date_cols = detect_date_columns(sample)

            # Load full CSV parsing detected date columns
            df = pd.read_csv(file_path, parse_dates=date_cols)
            
            # Register and write to DuckDB
            con.register('df_view', df)
            con.execute(f"CREATE OR REPLACE TABLE {table_name} AS SELECT * FROM df_view")
            print(f"Loaded: {table_name} (dates: {date_cols})")
        except Exception as e:
            print(f"Failed to load {file_name}: {e}")


In [ ]:
# FROM https://github.com/microsoft/mimic_sepsis

# 1. culture
# These correspond to blood/urine/CSF/sputum cultures etc
# There are 18 chartevent tables in the Mimic III database, one unsubscripted and 
# the others subscripted from 1 to 17. We use the unsubscripted one to create the 
# culture subtable. The remaining 17 are just partitions and should not be directly queried.
# The labels corresponding to the 51 itemids in the query below are:
"""
 Itemid | Label
-----------------------------------------------------
    938 | blood cultures
    941 | urine culture
    942 | BLOOD CULTURES
   2929 | sputum culture
   3333 | Blood Cultures
   4855 | Urine culture
   6035 | Urinalysis sent
   6043 | surface cultures
  70006 | ANORECTAL/VAGINAL CULTURE
  70011 | BLOOD CULTURE ( MYCO/F LYTIC BOTTLE)
  70012 | BLOOD CULTURE
  70013 | FLUID RECEIVED IN BLOOD CULTURE BOTTLES
  70014 | BLOOD CULTURE - NEONATE
  70016 | BLOOD CULTURE (POST-MORTEM)
  70024 | VIRAL CULTURE: R/O CYTOMEGALOVIRUS
  70037 | FOOT CULTURE
  70041 | VIRAL CULTURE:R/O HERPES SIMPLEX VIRUS
  70055 | POSTMORTEM CULTURE
  70057 | Rapid Respiratory Viral Screen & Culture
  70060 | Stem Cell - Blood Culture
  70063 | STERILITY CULTURE
  70075 | THROAT CULTURE
  70083 | VARICELLA-ZOSTER CULTURE
  80220 | AFB GROWN IN CULTURE; ADDITIONAL INFORMATION TO FOLLOW
 225401 | Blood Cultured
 225437 | CSF Culture
 225444 | Pan Culture
 225451 | Sputum Culture
 225454 | Urine Culture
 225722 | Arterial Line Tip Cultured
 225723 | CCO PAC Line Tip Cultured
 225724 | Cordis/Introducer Line Tip Cultured
 225725 | Dialysis Catheter Tip Cultured
 225726 | Tunneled (Hickman) Line Tip Cultured
 225727 | IABP Line Tip Cultured
 225728 | Midline Tip Cultured
 225729 | Multi Lumen Line Tip Cultured
 225730 | PA Catheter Line Tip Cultured
 225731 | Pheresis Catheter Line Tip Cultured
 225732 | PICC Line Tip Cultured
 225733 | Indwelling Port (PortaCath) Line Tip Cultured
 225734 | Presep Catheter Line Tip Cultured
 225735 | Trauma Line Tip Cultured
 225736 | Triple Introducer Line Tip Cultured
 225768 | Sheath Line Tip Cultured
 225814 | Stool Culture
 225816 | Wound Culture
 225817 | BAL Fluid Culture
 225818 | Pleural Fluid Culture
 226131 | ICP Line Tip Cultured
 227726 | AVA Line Tip Cultured
"""

culture_itemids = (
    6035, 3333, 938, 941, 942, 4855, 6043, 2929, 225401, 225437, 225444, 225451,
    225454, 225814, 225816, 225817, 225818, 225722, 225723, 225724, 225725,
    225726, 225727, 225728, 225729, 225730, 225731, 225732, 225733, 227726,
    70006, 70011, 70012, 70013, 70014, 70016, 70024, 70037, 70041, 225734,
    225735, 225736, 225768, 70055, 70057, 70060, 70063, 70075, 70083, 226131,
    80220
)

chartevents_query = f"""
SELECT subject_id, hadm_id, charttime AS culture_time, itemid
FROM chartevents
WHERE itemid IN {culture_itemids}
ORDER BY subject_id, hadm_id, charttime;
"""

chartevents_df = con.execute(chartevents_query).fetchdf()
display(chartevents_df.shape)
display(chartevents_df)

In [ ]:
microbiologyevents_query = """
SELECT subject_id, hadm_id, charttime AS culture_time
FROM microbiologyevents
WHERE charttime IS NOT NULL
ORDER BY subject_id, hadm_id, charttime;
"""

microbiologyevents_df = con.execute(microbiologyevents_query).fetchdf()

# Combine culture events
culture_events_df = pd.concat([chartevents_df, microbiologyevents_df])
display(culture_events_df.shape)
print('Unique Patients:')
display(culture_events_df['subject_id'].nunique())
print('Unique Admissions:')
display(culture_events_df['hadm_id'].nunique())
display(culture_events_df)


In [ ]:
# taken from: https://github.com/microsoft/mimic_sepsis/blob/main/preprocess.py
gsn_codes = (
    '002542','002543','007371','008873','008877','008879','008880','008935','008941',
    '008942','008943','008944','008983','008984','008990','008991','008992','008995',
    '008996','008998','009043','009046','009065','009066','009136','009137','009162',
    '009164','009165','009171','009182','009189','009213','009214','009218','009219',
    '009221','009226','009227','009235','009242','009263','009273','009284','009298',
    '009299','009310','009322','009323','009326','009327','009339','009346','009351',
    '009354','009362','009394','009395','009396','009509','009510','009511','009544',
    '009585','009591','009592','009630','013023','013645','013723','013724','013725',
    '014182','014500','015979','016368','016373','016408','016931','016932','016949',
    '018636','018637','018766','019283','021187','021205','021735','021871','023372',
    '023989','024095','024194','024668','025080','026721','027252','027465','027470',
    '029325','029927','029928','037042','039551','039806','040819','041798','043350',
    '043879','044143','045131','045132','046771','047797','048077','048262','048266',
    '048292','049835','050442','050443','051932','052050','060365','066295','067471'
)

gsn_codes_str = ','.join(f"'{code}'" for code in gsn_codes)

prescriptions_query = f"""
SELECT subject_id, hadm_id, startdate AS antibiotic_time, drug, gsn
FROM prescriptions
WHERE gsn IN ({gsn_codes_str})
ORDER BY subject_id, hadm_id, startdate;
"""

antibiotics_df = con.execute(prescriptions_query).fetchdf()
display(antibiotics_df.shape)
print('Unique Patients:')
display(antibiotics_df['subject_id'].nunique())
print('Unique Admissions:')
display(antibiotics_df['hadm_id'].nunique())
display(antibiotics_df)


In [ ]:
merged_df = pd.merge(
    culture_events_df,
    antibiotics_df,
    on=['subject_id', 'hadm_id'],
    how='inner',
    suffixes=('_culture', '_antibiotic')
)

# Convert time columns to datetime
merged_df['culture_time'] = pd.to_datetime(merged_df['culture_time'])
merged_df['antibiotic_time'] = pd.to_datetime(merged_df['antibiotic_time'])

# Calculate time difference in hours
merged_df['time_diff_hours'] = (merged_df['antibiotic_time'] - merged_df['culture_time']).dt.total_seconds() / 3600

# Filter for suspected infection criteria: antibiotic given within -24 to +72 hours of culture
suspected_infection_df = merged_df[
    (merged_df['time_diff_hours'] >= -24) & (merged_df['time_diff_hours'] <= 72)
]

# Determine the earliest suspected infection time per admission
suspected_infection_times = (
    suspected_infection_df
    .groupby(['subject_id', 'hadm_id'])['culture_time']
    .min()
    .reset_index()
    .rename(columns={'culture_time': 'suspected_infection_time'})
)

# Display the suspected infection times
print(suspected_infection_times)

In [ ]:
## daily SOFA score - based on: https://github.com/MIT-LCP/mimic-code/blob/main/mimic-iii/concepts/severityscores/sofa.sql
# Break it down by organ:
respiratory_sql = """
WITH resp_data AS (
  SELECT
    ce.subject_id,
    ce.hadm_id,
    ce.icustay_id,
    DATE_DIFF('day', ie.intime, ce.charttime) AS icu_day,
    ce.charttime,
    CASE WHEN ce.itemid = 50821 THEN ce.valuenum END AS pao2,
    CASE WHEN ce.itemid IN (223835, 3420) THEN ce.valuenum END AS fio2
  FROM chartevents ce
  JOIN icustays ie ON ce.icustay_id = ie.icustay_id
  WHERE ce.itemid IN (50821, 223835, 3420)
    AND ce.valuenum IS NOT NULL
    AND ce.charttime BETWEEN ie.intime AND ie.outtime
),
merged_resp AS (
  SELECT subject_id, hadm_id, icustay_id, icu_day,
         MAX(pao2) AS pao2, MAX(fio2) AS fio2
  FROM resp_data
  GROUP BY subject_id, hadm_id, icustay_id, icu_day
  HAVING MAX(pao2) IS NOT NULL AND MAX(fio2) IS NOT NULL AND CAST(MAX(fio2) AS DOUBLE) > 0
)
SELECT subject_id, hadm_id, icustay_id, icu_day, pao2, fio2,
       ROUND(CAST(pao2 AS DOUBLE) / CAST(fio2 AS DOUBLE), 1) AS pf_ratio,
       CASE
         WHEN CAST(pao2 AS DOUBLE) / CAST(fio2 AS DOUBLE) > 400 THEN 0
         WHEN CAST(pao2 AS DOUBLE) / CAST(fio2 AS DOUBLE) > 300 THEN 1
         WHEN CAST(pao2 AS DOUBLE) / CAST(fio2 AS DOUBLE) > 200 THEN 2
         WHEN CAST(pao2 AS DOUBLE) / CAST(fio2 AS DOUBLE) > 100 THEN 3
         ELSE 4
       END AS respiratory_sofa
FROM merged_resp;
"""


coagulation_sql = """
WITH platelets AS (
  SELECT le.subject_id, le.hadm_id, ie.icustay_id,
         DATE_DIFF('day', ie.intime, le.charttime) AS icu_day,
         le.charttime, le.valuenum AS platelet
  FROM labevents le
  JOIN icustays ie ON le.subject_id = ie.subject_id
  WHERE le.itemid = 51265
    AND le.valuenum IS NOT NULL
    AND le.charttime BETWEEN ie.intime AND ie.outtime
),
daily_platelets AS (
  SELECT subject_id, hadm_id, icustay_id, icu_day,
         MIN(platelet) AS min_platelet
  FROM platelets
  GROUP BY subject_id, hadm_id, icustay_id, icu_day
)
SELECT subject_id, hadm_id, icustay_id, icu_day, min_platelet,
       CASE
         WHEN min_platelet >= 150 THEN 0
         WHEN min_platelet >= 100 THEN 1
         WHEN min_platelet >= 50 THEN 2
         WHEN min_platelet >= 20 THEN 3
         ELSE 4
       END AS coagulation_sofa
FROM daily_platelets;
"""

# DuckDB SQL query for Liver SOFA score (bilirubin-based)
liver_sql = """
WITH bilirubin_labs AS (
  SELECT
    le.subject_id,
    le.hadm_id,
    ie.icustay_id,
    DATE_DIFF('day', ie.intime, le.charttime) AS icu_day,
    le.charttime,
    le.valuenum AS bilirubin
  FROM labevents le
  JOIN icustays ie ON le.subject_id = ie.subject_id
  WHERE le.itemid = 50885
    AND le.valuenum IS NOT NULL
    AND le.charttime BETWEEN ie.intime AND ie.outtime
),
daily_bilirubin AS (
  SELECT
    subject_id,
    hadm_id,
    icustay_id,
    icu_day,
    MAX(bilirubin) AS max_bilirubin
  FROM bilirubin_labs
  GROUP BY subject_id, hadm_id, icustay_id, icu_day
)
SELECT
  subject_id,
  hadm_id,
  icustay_id,
  icu_day,
  max_bilirubin,
  CASE
    WHEN max_bilirubin < 1.2 THEN 0
    WHEN max_bilirubin < 2.0 THEN 1
    WHEN max_bilirubin < 6.0 THEN 2
    WHEN max_bilirubin < 12.0 THEN 3
    ELSE 4
  END AS liver_sofa
FROM daily_bilirubin
ORDER BY subject_id, icustay_id, icu_day;
"""

liver_sql = """
WITH bilirubin_labs AS (
  SELECT
    le.subject_id,
    le.hadm_id,
    ie.icustay_id,
    DATE_DIFF('day', ie.intime, le.charttime) AS icu_day,
    le.charttime,
    le.valuenum AS bilirubin
  FROM labevents le
  JOIN icustays ie ON le.subject_id = ie.subject_id
  WHERE le.itemid = 50885
    AND le.valuenum IS NOT NULL
    AND le.charttime BETWEEN ie.intime AND ie.outtime
),
daily_bilirubin AS (
  SELECT
    subject_id,
    hadm_id,
    icustay_id,
    icu_day,
    MAX(bilirubin) AS max_bilirubin
  FROM bilirubin_labs
  GROUP BY subject_id, hadm_id, icustay_id, icu_day
)
SELECT
  subject_id,
  hadm_id,
  icustay_id,
  icu_day,
  max_bilirubin,
  CASE
    WHEN max_bilirubin < 1.2 THEN 0
    WHEN max_bilirubin < 2.0 THEN 1
    WHEN max_bilirubin < 6.0 THEN 2
    WHEN max_bilirubin < 12.0 THEN 3
    ELSE 4
  END AS liver_sofa
FROM daily_bilirubin
ORDER BY subject_id, icustay_id, icu_day;
"""

# MAP-based SOFA SQL
map_sql = """
WITH map_events AS (
  SELECT
    ce.subject_id,
    ce.hadm_id,
    ce.icustay_id,
    DATE_DIFF('day', ie.intime, ce.charttime) AS icu_day,
    ce.charttime,
    ce.valuenum AS map
  FROM chartevents ce
  JOIN icustays ie ON ce.icustay_id = ie.icustay_id
  WHERE ce.itemid IN (220052, 51)
    AND ce.valuenum IS NOT NULL
    AND ce.charttime BETWEEN ie.intime AND ie.outtime
),
daily_map AS (
  SELECT
    subject_id,
    hadm_id,
    icustay_id,
    icu_day,
    MIN(map) AS min_map
  FROM map_events
  GROUP BY subject_id, hadm_id, icustay_id, icu_day
)
SELECT
  subject_id,
  hadm_id,
  icustay_id,
  icu_day,
  min_map,
  CASE
    WHEN CAST(min_map AS DOUBLE) < 70 THEN 1
    ELSE 0
  END AS map_score
FROM daily_map
ORDER BY subject_id, icustay_id, icu_day;
"""

vasopressor_sql = """
WITH vasopressors AS (
  SELECT
    imv.subject_id,
    imv.hadm_id,
    imv.icustay_id,
    DATE_DIFF('day', ie.intime, imv.starttime) AS icu_day,
    imv.starttime,
    di.label,
    CASE
      WHEN LOWER(di.label) LIKE '%dopamine%' THEN 'dopamine'
      WHEN LOWER(di.label) LIKE '%dobutamine%' THEN 'dobutamine'
      WHEN LOWER(di.label) LIKE '%epinephrine%' THEN 'epinephrine'
      WHEN LOWER(di.label) LIKE '%norepinephrine%' THEN 'norepinephrine'
    END AS vasopressor,
    imv.rate AS dose
  FROM inputevents_mv imv
  JOIN icustays ie ON imv.icustay_id = ie.icustay_id
  JOIN d_items di ON imv.itemid = di.itemid
  WHERE imv.rate IS NOT NULL
    AND imv.starttime BETWEEN ie.intime AND ie.outtime
    AND (
      LOWER(di.label) LIKE '%dopamine%' OR
      LOWER(di.label) LIKE '%dobutamine%' OR
      LOWER(di.label) LIKE '%epinephrine%' OR
      LOWER(di.label) LIKE '%norepinephrine%'
    )
),

daily_max_pressors AS (
  SELECT
    subject_id,
    hadm_id,
    icustay_id,
    icu_day,
    vasopressor,
    MAX(dose) AS max_dose
  FROM vasopressors
  GROUP BY subject_id, hadm_id, icustay_id, icu_day, vasopressor
),

sofa_scores AS (
  SELECT
    subject_id,
    hadm_id,
    icustay_id,
    icu_day,
    MAX(CASE
      WHEN vasopressor = 'dobutamine' THEN 2
      WHEN vasopressor = 'dopamine' AND max_dose <= 5 THEN 2
      WHEN vasopressor = 'dopamine' AND max_dose <= 15 THEN 3
      WHEN vasopressor = 'dopamine' AND max_dose > 15 THEN 4
      WHEN vasopressor IN ('epinephrine', 'norepinephrine') AND max_dose <= 0.1 THEN 3
      WHEN vasopressor IN ('epinephrine', 'norepinephrine') AND max_dose > 0.1 THEN 4
    END) AS vasopressor_score
  FROM daily_max_pressors
  GROUP BY subject_id, hadm_id, icustay_id, icu_day
)

SELECT * FROM sofa_scores
ORDER BY subject_id, icustay_id, icu_day;
"""

gcs_sql = """
WITH gcs_data AS (
  SELECT
    ce.subject_id,
    ce.hadm_id,
    ce.icustay_id,
    DATE_DIFF('day', ie.intime, ce.charttime) AS icu_day,
    ce.charttime,
    ce.valuenum AS gcs
  FROM chartevents ce
  JOIN icustays ie ON ce.icustay_id = ie.icustay_id
  WHERE ce.itemid = 223900
    AND ce.valuenum IS NOT NULL
    AND ce.charttime BETWEEN ie.intime AND ie.outtime
),

gcs_per_day AS (
  SELECT
    subject_id,
    hadm_id,
    icustay_id,
    icu_day,
    MIN(gcs) AS min_gcs
  FROM gcs_data
  GROUP BY subject_id, hadm_id, icustay_id, icu_day
)

SELECT
  subject_id,
  hadm_id,
  icustay_id,
  icu_day,
  min_gcs,
  CASE
    WHEN min_gcs = 15 THEN 0
    WHEN min_gcs >= 13 THEN 1
    WHEN min_gcs >= 10 THEN 2
    WHEN min_gcs >= 6 THEN 3
    ELSE 4
  END AS cns_sofa
FROM gcs_per_day
ORDER BY subject_id, icustay_id, icu_day;"""

cns_sql = """
WITH gcs_data AS (
  SELECT
    ce.subject_id,
    ce.hadm_id,
    ce.icustay_id,
    DATE_DIFF('day', ie.intime, ce.charttime) AS icu_day,
    ce.charttime,
    ce.valuenum AS gcs
  FROM chartevents ce
  JOIN icustays ie ON ce.icustay_id = ie.icustay_id
  WHERE ce.itemid = 223900
    AND ce.valuenum IS NOT NULL
    AND ce.charttime BETWEEN ie.intime AND ie.outtime
),
gcs_per_day AS (
  SELECT
    subject_id,
    hadm_id,
    icustay_id,
    icu_day,
    MIN(gcs) AS min_gcs
  FROM gcs_data
  GROUP BY subject_id, hadm_id, icustay_id, icu_day
)
SELECT
  subject_id,
  hadm_id,
  icustay_id,
  icu_day,
  min_gcs,
  CASE
    WHEN CAST(min_gcs AS DOUBLE) = 15 THEN 0
    WHEN CAST(min_gcs AS DOUBLE) >= 13 THEN 1
    WHEN CAST(min_gcs AS DOUBLE) >= 10 THEN 2
    WHEN CAST(min_gcs AS DOUBLE) >= 6 THEN 3
    ELSE 4
  END AS cns_sofa
FROM gcs_per_day
ORDER BY subject_id, icustay_id, icu_day;
"""

renal_sql = """
WITH creatinine_data AS (
  SELECT
    le.subject_id,
    le.hadm_id,
    ie.icustay_id,
    DATE_DIFF('day', ie.intime, le.charttime) AS icu_day,
    le.charttime,
    le.valuenum AS creatinine
  FROM labevents le
  JOIN icustays ie ON le.subject_id = ie.subject_id
  WHERE le.itemid = 50912
    AND le.valuenum IS NOT NULL
    AND le.charttime BETWEEN ie.intime AND ie.outtime
),
creatinine_per_day AS (
  SELECT
    subject_id,
    hadm_id,
    icustay_id,
    icu_day,
    MAX(creatinine) AS max_creatinine
  FROM creatinine_data
  GROUP BY subject_id, hadm_id, icustay_id, icu_day
)
SELECT
  subject_id,
  hadm_id,
  icustay_id,
  icu_day,
  max_creatinine,
  CASE
    WHEN max_creatinine < 1.2 THEN 0
    WHEN max_creatinine < 2.0 THEN 1
    WHEN max_creatinine < 3.5 THEN 2
    WHEN max_creatinine < 5.0 THEN 3
    ELSE 4
  END AS renal_sofa
FROM creatinine_per_day
ORDER BY subject_id, icustay_id, icu_day;
"""


In [ ]:
# Create temporary views from component queries
# con.execute(f"CREATE TEMP VIEW respiratory AS {respiratory_sql}")
# con.execute(f"CREATE TEMP VIEW coagulation AS {coagulation_sql}")
# con.execute(f"CREATE TEMP VIEW liver AS {liver_sql}")
# con.execute(f"CREATE TEMP VIEW map_scores AS {map_sql}")
# con.execute(f"CREATE TEMP VIEW vasopressors AS {vasopressor_sql}")
# con.execute(f"CREATE TEMP VIEW cns AS {cns_sql}")
# con.execute(f"CREATE TEMP VIEW renal AS {renal_sql}")

# Final SOFA join
# SQL to join SOFA components with icu_intime to get datetime context
sofa_with_time_sql = """
WITH cardiovascular AS (
  SELECT
    COALESCE(CAST(m.subject_id AS BIGINT), CAST(v.subject_id AS BIGINT)) AS subject_id,
    COALESCE(CAST(m.hadm_id AS BIGINT), CAST(v.hadm_id AS BIGINT)) AS hadm_id,
    COALESCE(CAST(m.icustay_id AS BIGINT), CAST(v.icustay_id AS BIGINT)) AS icustay_id,
    COALESCE(m.icu_day, v.icu_day) AS icu_day,
    GREATEST(
      COALESCE(CAST(m.map_score AS DOUBLE), 0),
      COALESCE(CAST(v.vasopressor_score AS DOUBLE), 0)
    ) AS cardiovascular_sofa
  FROM map_scores m
  FULL OUTER JOIN vasopressors v
    ON m.subject_id = v.subject_id
    AND m.icustay_id = v.icustay_id
    AND m.icu_day = v.icu_day
),
daily_sofa AS (
  SELECT
    COALESCE(CAST(r.subject_id AS BIGINT), CAST(c.subject_id AS BIGINT), CAST(l.subject_id AS BIGINT), CAST(cv.subject_id AS BIGINT), CAST(n.subject_id AS BIGINT), CAST(g.subject_id AS BIGINT)) AS subject_id,
    COALESCE(CAST(r.hadm_id AS BIGINT), CAST(c.hadm_id AS BIGINT), CAST(l.hadm_id AS BIGINT), CAST(cv.hadm_id AS BIGINT), CAST(n.hadm_id AS BIGINT), CAST(g.hadm_id AS BIGINT)) AS hadm_id,
    COALESCE(CAST(r.icustay_id AS BIGINT), CAST(c.icustay_id AS BIGINT), CAST(l.icustay_id AS BIGINT), CAST(cv.icustay_id AS BIGINT), CAST(n.icustay_id AS BIGINT), CAST(g.icustay_id AS BIGINT)) AS icustay_id,
    COALESCE(r.icu_day, c.icu_day, l.icu_day, cv.icu_day, n.icu_day, g.icu_day) AS icu_day,
    CAST(respiratory_sofa AS DOUBLE) AS respiratory_sofa,
    CAST(coagulation_sofa AS DOUBLE) AS coagulation_sofa,
    CAST(liver_sofa AS DOUBLE) AS liver_sofa,
    CAST(cardiovascular_sofa AS DOUBLE) AS cardiovascular_sofa,
    CAST(cns_sofa AS DOUBLE) AS cns_sofa,
    CAST(renal_sofa AS DOUBLE) AS renal_sofa,
    COALESCE(CAST(respiratory_sofa AS DOUBLE), 0)
      + COALESCE(CAST(coagulation_sofa AS DOUBLE), 0)
      + COALESCE(CAST(liver_sofa AS DOUBLE), 0)
      + COALESCE(CAST(cardiovascular_sofa AS DOUBLE), 0)
      + COALESCE(CAST(cns_sofa AS DOUBLE), 0)
      + COALESCE(CAST(renal_sofa AS DOUBLE), 0) AS total_sofa
  FROM respiratory r
  FULL OUTER JOIN coagulation c
    ON r.subject_id = c.subject_id AND r.icustay_id = c.icustay_id AND r.icu_day = c.icu_day
  FULL OUTER JOIN liver l
    ON r.subject_id = l.subject_id AND r.icustay_id = l.icustay_id AND r.icu_day = l.icu_day
  FULL OUTER JOIN cardiovascular cv
    ON r.subject_id = cv.subject_id AND r.icustay_id = cv.icustay_id AND r.icu_day = cv.icu_day
  FULL OUTER JOIN cns g
    ON r.subject_id = g.subject_id AND r.icustay_id = g.icustay_id AND r.icu_day = g.icu_day
  FULL OUTER JOIN renal n
    ON r.subject_id = n.subject_id AND r.icustay_id = n.icustay_id AND r.icu_day = n.icu_day
)
SELECT
  ds.*,
  icu.intime AS icu_intime
FROM daily_sofa ds
LEFT JOIN icustays icu
  ON ds.subject_id = icu.subject_id
 AND ds.hadm_id = icu.hadm_id
 AND ds.icustay_id = icu.icustay_id
"""

# Execute and return sofa_df
sofa_df = con.execute(sofa_with_time_sql).fetchdf()


In [ ]:
# now export sofa_df 
sofa_df.to_csv('./data/sofa_df.csv', index=False)

In [ ]:
sofa_df['delta_sofa'] = sofa_df.groupby(['subject_id', 'icustay_id'])['total_sofa'].diff()
sofa_df['sofa_increase_2plus'] = sofa_df['delta_sofa'] >= 2


In [ ]:
# Example: adding sofa_day_time = icu_intime + icu_day (in days)
sofa_df['icu_intime'] = pd.to_datetime(sofa_df['icu_intime'])
sofa_df['sofa_day_time'] = sofa_df['icu_intime'] + pd.to_timedelta(sofa_df['icu_day'], unit='D')

# Cast both sides to int64
sofa_df['subject_id'] = sofa_df['subject_id'].astype('int64')
sofa_df['hadm_id'] = sofa_df['hadm_id'].astype('int64')

suspected_infection_times['subject_id'] = suspected_infection_times['subject_id'].astype('int64')
suspected_infection_times['hadm_id'] = suspected_infection_times['hadm_id'].astype('int64')

merged = pd.merge(
    sofa_df,
    suspected_infection_times,
    on=['subject_id', 'hadm_id'],
    how='inner'
)

# Calculate time from infection to SOFA day
merged['hours_from_infection'] = (
    merged['sofa_day_time'] - pd.to_datetime(merged['suspected_infection_time'])
).dt.total_seconds() / 3600
windowed = merged[
    (merged['hours_from_infection'] >= -48) &
    (merged['hours_from_infection'] <= 24)
].copy()

# For each ICU stay, compute delta SOFA vs. minimum in window
windowed['min_sofa'] = windowed.groupby(['subject_id', 'icustay_id'])['total_sofa'].transform('min')
windowed['delta_sofa'] = windowed['total_sofa'] - windowed['min_sofa']

# Flag patients with ≥2 point rise
windowed['sepsis_flag'] = windowed['delta_sofa'] >= 2

sepsis_onsets = (
    windowed[windowed['sepsis_flag']]
    .groupby(['subject_id', 'hadm_id', 'icustay_id'])['sofa_day_time']
    .min()
    .reset_index()
    .rename(columns={'sofa_day_time': 'sepsis_onset_time'})
)


In [ ]:
sepsis_onsets